In [1]:
import keras
from keras import backend as K
import os
from importlib import reload

def set_keras_backend(backend):

    if K.backend() != backend:
        os.environ['KERAS_BACKEND'] = backend
        reload(K)
        assert K.backend() == backend

set_keras_backend("tensorflow")

Using Theano backend.
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
Using TensorFlow backend.


In [2]:
import numpy

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [3]:
"""with open(('wonderland.txt'), 'rb') as document:
  response = document_conversion.convert_document(document=document, config=config)"""

"with open(('wonderland.txt'), 'rb') as document:\n  response = document_conversion.convert_document(document=document, config=config)"

In [4]:
text = "wonderland.txt"
raw_text = open(text, encoding="utf8").read()
raw_text = raw_text.lower()

In [5]:
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [16]:
n_chars = len(raw_text)
n_vocab = len(chars)
n_vocab

46

In [15]:
seq_length = 100
XX = []
YY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    XX.append([char_to_int[char] for char in seq_in])
    YY.append(char_to_int[seq_out])
n_patterns = len(XX)
n_patterns

144225

In [8]:
X = numpy.reshape(XX, (n_patterns, seq_length, 1))
X = X / float(n_vocab)
y = np_utils.to_categorical(YY)

In [9]:
import multiprocessing
print(multiprocessing.cpu_count()) 

4


In [10]:
import multi_gpu as mg

In [11]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model = mg.make_parallel(model, multiprocessing.cpu_count())
model.compile(loss='categorical_crossentropy', optimizer='adam')

C:\Users\Palash Goyal\Documents\DeepLearning\Blog\multi_gpu.py:44: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  merged.append(merge(outputs, mode='concat', concat_axis=0))
C:\Users\Palash Goyal\AppData\Roaming\Python\Python35\site-packages\keras-2.0.5-py3.5.egg\keras\legacy\layers.py:460: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
C:\Users\Palash Goyal\Documents\DeepLearning\Blog\multi_gpu.py:46: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`
  return Model(input=model.inputs, output=merged)


In [1]:
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
model.fit(X, y, epochs=20, batch_size=64, callbacks=callbacks_list)

In [20]:
import sys
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [ ]:
import sys
filename = "weights-improvement-47-1.6669.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')
start = numpy.random.randint(0, len(XX)-1)
pattern = XX[start]
print "Start:"
print "\"", ''.join([int_to_char[value] for value in pattern]), "\""
for i in range(100):
                x = numpy.reshape(pattern, (1, len(pattern), 1))
                x = x / float(n_vocab)
                prediction = model.predict(x, verbose=0)
                index = numpy.argmax(prediction)
                result = int_to_char[index]
                seq_in = [int_to_char[value] for value in pattern]
                sys.stdout.write(result)
                pattern.append(index)
                pattern = pattern[1:len(pattern)]
print "\nEnd"

Start:
" tried
hedges,’ the pigeon went on, without attending to her; ‘but those
serpents! there’s no p "
ice the shattee onte the toaas then!�
the aaded inoa